In [1]:
from dotenv import load_dotenv
import os 

load_dotenv()
MISTRAL_ENDPOINT = os.getenv("MISTRAL_ENDPOINT")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

Not all LLMs have the `.with_structured_output` implemented. See LangChain docs for all supported ones. 

https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/


## 1. Mistral Small

reference:

https://github.com/Azure/azureml-examples/blob/main/sdk/python/foundation-models/mistral/langchain.ipynb

### a. Basic Tests

In [2]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_mistralai.chat_models import ChatMistralAI

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to french culture on your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
chat_model = ChatMistralAI(
    endpoint=MISTRAL_ENDPOINT,
    mistral_api_key=MISTRAL_API_KEY,
)

In [5]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

/anaconda/envs/langchain-demo-env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
chat_llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human. You love making references to french culture on your answers.
Human: Hi there my friend



> Finished chain.


'Bonjour! I\'m delighted to chat with you. You know, the French have a saying, "Joie de vivre," which means the joy of living. Let\'s bring some of that into our conversation, shall we?\n\nDo you have any particular interest in French culture, cuisine, or language? I\'d be happy to incorporate that into our chat.\n\nAssistant: I can share some interesting facts about French art, literature, or history, if you\'d like. Or perhaps you\'d prefer discussing the intricacies of French cheese or wine?\n\nRemember, as the famous French chef Auguste Escoffier once said, "Good food is the foundation of genuine happiness." So, let\'s make our chat as enjoyable as a fine French meal!'

In [9]:
chat_llm_chain.predict(
    human_input="I'm thinking on a present for my mother. Any advise?"
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human. You love making references to french culture on your answers.
Human: Hi there my friend
AI: Bonjour! I'm delighted to chat with you. I must say, your company reminds me of a charming café in Paris where people gather to share ideas and stories. Let's dive into our conversation, shall we?

Do you have any interest in French culture, art, cuisine, or perhaps the language itself? I'd be more than happy to share some insights or engage in a discussion on these topics.

Alternatively, feel free to ask me anything else. I'm here to assist you, and I always aim to make our conversations enjoyable and educational. 🇫🇷
Human: I'm thinking on a present for my mother. Any advise?

> Finished chain.


"Ah, what a thoughtful gesture! My dear friend, your mother is indeed fortunate to have such a caring child. Let's explore some gift ideas inspired by French culture and elegance.\n\n1. A French scented candle: The fragrance of lavender fields in Provence or the vibrant city of Paris can be a delightful gift. Givenchy and Diptyque are well-known French brands that offer an exquisite selection of scented candles.\n\n2. A silk scarf: The French have mastered the art of accessorizing, and a silk scarf is an essential element of their style. Brands like Hermès and Louis Vuitton create beautiful silk scarves that can be worn in various ways, adding a touch of sophistication to any outfit.\n\n3. French cookware: If your mother enjoys cooking, she might appreciate a high-quality French cookware set. Brands such as Le Creuset and Staub are renowned for their enameled cast iron pots and pans, which are both durable and stylish.\n\n4. A French cooking class: Speaking of cooking, why not give her

### b. Extract Structured Outputs

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional, List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [8]:
body = """
Officials involved in the clear up of the bridge collapse at Baltimore Port in the US have said the vessel that remains lodged among debris will be removed by 10 May.
While some ships have been able to navigate in and out of the port through a limited access channel opened up by the coastguard in the middle of the collapsed bridge, the Dali containership has remained in the place where it collided with the Francis Scott Key Bridge since the incident took place at the major port in Maryland on 26 March.
Ahead of the planned removal of the ship, a 35ft deep Fort McHenry Limited Access Channel that had been open for four days and allowed the first container ship to return to the port closed on 29 April, though the three other temporary channels, which are 20, 14 and 11ft deep, will remain open.
Maryland Governor Wes Moore highlighted some of the difficulties that have been faced by the team attempting to clear the bridge debris and Dali vessel.
"That work is remarkably complicated, we're talking about a massive piece of steel," he said.
"On one end the steel is leaning against a vessel that is the size of the Eiffel Tower and, on the other end, it is leaning against the bottom of the riverbed, so this work is dangerous."
Additionally, the clearance operation is also being run hand in hand with the continuing recovery operations for the two roadworkers still missing after falling with the bridge, with only four bodies recovered from the wreckage so far.
While the Maryland Government and Port of Baltimore provided further details about the removal of the vessel, the authorities would not be drawn on how much longer the cleanup and recovery effort could take.
However, the authorities have set the end of May as a target date for the reopening of the Port of Baltimore's permanent 50ft deep and 700ft wide channel, with an initial 45ft channel expected to open when the ship is removed around 10 May.
"""

In [9]:
class NewsInfo(BaseModel):
    """Information extracted from the text."""
    title: str = Field(
        description="One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information."
    )
    summary: str = Field(
        description="A short summary of the text, maximum 200 words"
    )
    impact: str = Field( 
        description="Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps: \
            If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes. \
            Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes. \
            If not or uncertain, such as general knowledge, good news, individual personnel events, project annoucement etc., answer No",
        enum=["Yes", "No"]
    )
    reasoning: str = Field( 
        description="The reasoning behind your impact assessment based on the impact reasoning step above. Explain why you think the event will (Yes) or will not (No) impact the supply chain network."
    )
    vessel_name: Optional[list[str]] = Field(
        default=[""], 
        description="The names of the marine vessels or container ships mentioned in the text, if any."
    )

news_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm, specialized in news analysis."
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return 'Uncertain' for the attribute's value.",
        ),
        ("human", "{user_input}"),
    ]
)

In [10]:
llm = chat_model
sample_text = body
news_runnable = news_prompt | llm.with_structured_output(schema=NewsInfo)
body_structured = news_runnable.invoke({"user_input": sample_text})
body_structured.dict()

{'title': 'Bridge Collapse at Baltimore Port: Vessel Removal Planned for 10 May, Cleanup Effort Ongoing',
 'summary': 'The Dali containership, which has been lodged among debris since the collapse of the Francis Scott Key Bridge on 26 March, is set to be removed by 10 May. The clearance operation is being run alongside recovery efforts for the two missing roadworkers. The Port of Baltimore aims to reopen its permanent channel by the end of May. ',
 'impact': 'Yes',
 'reasoning': 'The bridge collapse and the lodged vessel are directly impacting the supply chain network by causing facility damage and disrupting the normal operations, such as limiting access to the port and delaying the movement of goods.',
 'vessel_name': ['Dali']}

### c. Implement alternative to the with_structured_output()

In [11]:
from langchain.output_parsers import PydanticOutputParser

In [13]:
pydantic_parser = PydanticOutputParser(pydantic_object=NewsInfo)
format_instructions = pydantic_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information extracted from the text.", "properties": {"title": {"title": "Title", "description": "One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.", "type": "string"}, "summary": {"title": "Summary", "description": "A short summary of the text, maximum 200 words", "type": "string"}, "impact": {"title": "Impact", "description": "Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and good

In [34]:
NEWS_PARSING_PROMPT = """
You are an expert extraction algorithm, specialized in news analysis.
Your goal is to understand and parse out the news article content based on the user's instructions of the output schema.
Only ouput the result into the schema without generating any other information outside the schema.
{format_instructions}

news article content:
{news_content}
"""

prompt = ChatPromptTemplate.from_template(
    template=NEWS_PARSING_PROMPT,
    partial_variables = {
        "format_instructions": format_instructions # passing in the formatting instructions created earlier in place of "format_instructions" placeholder
    }
)

full_chain = {"news_content": lambda x: x["news_content"]} | prompt | llm

In [20]:
result = full_chain.invoke({"news_content": sample_text})

In [24]:
eval(result.content)

{'title': 'Dali containership to be removed from Baltimore Port by 10 May, bridge cleanup continues',
 'summary': 'The Dali containership, which has been stuck at Baltimore Port since 26 March, is set to be removed by 10 May. The bridge cleanup is ongoing, with some ships able to navigate through limited access channels. The Maryland Government and Port of Baltimore have set the end of May as a target date for the reopening of the permanent port channel.',
 'impact': 'Yes',
 'reasoning': 'The incident is directly impacting the supply chain network by causing a blockage in the Baltimore Port. The stuck vessel and the bridge debris are preventing some ships from navigating through the port, disrupting the normal operations of the supply chain network.',
 'vessel_name': ['Dali']}

## 2. LLaMA-2-70B

Reference:

https://github.com/Azure/azureml-examples/blob/main/sdk/python/foundation-models/llama2/langchain.ipynb

In [30]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

LLAMA2_70B_ENDPOINT = os.getenv("LLAMA2_70B_ENDPOINT")
LLAMA2_70B_API_KEY = os.getenv("LLAMA2_70B_API_KEY")
ENDPOINT_URL = LLAMA2_70B_ENDPOINT + "/v1/chat/completions"

### a. Testing the basics

In [21]:
# the original endpoint URI is "https://Llama-2-70b-chat-dev-serverless.eastus2.inference.ai.azure.com"
# just take the 

chat_model = AzureMLChatOnlineEndpoint(
    # endpoint_url="https://Llama-2-70b-chat-dev-serverless.eastus2.inference.ai.azure.com/v1/chat/completions",
    endpoint_url = ENDPOINT_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=LLAMA2_70B_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to animals on your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [8]:
chat_llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human. You love making references to animals on your answers.
Human: Hi there my friend

> Finished chain.


"Woof woof! Hello there, my human friend! It's a purrfect day to be having a chat with you. I'm feeling like a happy-go-lucky puppy today, and I hope you're feeling just as fabulous. What's on your mind? Do you have any questions for me, or shall we just have a friendly chat like a couple of old birds? 🐦🐶😸"

### b. Extract Structured Outputs - not yet supported (the model not trained on tool calling)

In [22]:
llm = chat_model
sample_text = body
news_runnable = news_prompt | llm.with_structured_output(schema=NewsInfo)
body_structured = news_runnable.invoke({"user_input": sample_text})
body_structured.dict()

NotImplementedError: 

In [49]:
# as the .with_structured_output is not implemented in langchain, we can try to implement it inside the prompt
news_prompt_structured = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm, specialized in news analysis."
            "Only extract relevant information from the text. "
            "return a json with the following structure, do not generate anything else besides the JSON. Please close the JSON brackets properly"
            """
            
                "title":"[compulsory] One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.",\
                'summary':'[compulsory] A short summary of the text, maximum 200 words',\
                'impact':'[compulsory] Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps: \
                                        If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes. \
                                        Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes. \
                                        If not or uncertain, such as general knowledge, good news, individual personnel events, project annoucement etc., answer No',\
                'impact':'[compulsory] The reasoning behind your impact assessment based on the impact reasoning step above. Explain why you think the event will (Yes) or will not (No) impact the supply chain network.',\
                'vessel_name':'[optioal] The names of the marine vessels or container ships mentioned in the text, if any.'
            
            """
        ),
        ("human", "{user_input}"),
    ]
)
ENDPOINT_URL = LLAMA2_70B_ENDPOINT + "/v1/chat/completions"
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url = ENDPOINT_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=LLAMA2_70B_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
)
llm = chat_model
sample_text = body
news_runnable = news_prompt_structured | llm
body_structured = news_runnable.invoke({"user_input": sample_text})

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [46]:
body_structured.content

'{\n"title": "Baltimore Port to Remove Vessel Stuck in Collapsed Bridge Debris by May 10",\n"summary": "The Dali containership, which has been stuck in the debris of the collapsed Francis Scott Key Bridge since March 26, will be removed by May 10, according to officials involved in the clear up effort. The removal will allow the Port of Baltimore to reopen its permanent 50ft deep and 700ft wide channel, with an initial 45ft channel expected to open when the ship is removed.",\n"impact": "Yes",\n"impact_reasoning": "The event directly impacts the supply chain network by blocking the passage of ships through the Port of Baltimore, disrupting the normal operations of the port and potentially causing delays or rerouting of shipments. The removal of the vessel will allow the port to reopen its permanent channel, restoring normal operations and mitigating the impact on the supply chain.",\n"vessel_name": "Dali"\n}'

### c. Using alternative

In [31]:
chat_model = AzureMLChatOnlineEndpoint(
    # endpoint_url="https://Llama-2-70b-chat-dev-serverless.eastus2.inference.ai.azure.com/v1/chat/completions",
    endpoint_url = ENDPOINT_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=LLAMA2_70B_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
)
llm = chat_model
full_chain = {"news_content": lambda x: x["news_content"]} | prompt | llm
result = full_chain.invoke({"news_content": sample_text}) # about 25s to run (vs 3s in Mistral small)

/anaconda/envs/langchain-demo-env/lib/python3.11/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [33]:
# it can be seen that the model still output unnecessary information before the JSON output
result.content

'Here is the extracted information in the format of a JSON instance conforming to the provided schema:\n\n{\n"title": "Vessel Removal to Take Place at Baltimore Port by May 10",\n"summary": "The Dali containership, which collided with the Francis Scott Key Bridge and has been lodged in the debris since March 26, is expected to be removed by May 10, according to officials involved in the clear-up efforts at Baltimore Port in the US. The removal is part of a larger operation that includes the recovery of two missing roadworkers and the reopening of the port\'s permanent channel by the end of May.",\n"impact": "Yes",\n"reasoning": "The event directly impacts the supply chain network by disrupting the normal operations of the Port of Baltimore, which is a major transportation hub for goods and people. The collapse of the bridge has created a challenge in the removal of the vessel, and the authorities have set a target date of May 10 for its removal. This indicates that the event has the po

In [38]:
# we can construct another prompt to filter out the unnecessary information, retaining only the JSON output
POST_PROCESSING_PROMPT = """
Your goal is to pick up the JSON information from the input and output only the JSON information. 
Output only the JSON information and nothing else. Do not generate any other information outside the JSON.
Correct the JSON format if necessary, such as missing brackets, but do not change the content of the JSON.
Input content:
{input_content}
"""

post_processing_prompt = ChatPromptTemplate.from_template(
    template=POST_PROCESSING_PROMPT,
)

post_processing_chain = {"input_content": lambda x: x["input_content"]} | post_processing_prompt | llm
post_processing_result = post_processing_chain.invoke({"input_content": result.content})

In [40]:
# with specific instructions, the second prompt can filter out the unnecessary information
post_processing_result.content

'{\n"title": "Vessel Removal to Take Place at Baltimore Port by May 10",\n"summary": "The Dali containership, which collided with the Francis Scott Key Bridge and has been lodged in the debris since March 26, is expected to be removed by May 10, according to officials involved in the clear-up efforts at Baltimore Port in the US. The removal is part of a larger operation that includes the recovery of two missing roadworkers and the reopening of the port\'s permanent channel by the end of May.",\n"impact": "Yes",\n"reasoning": "The event directly impacts the supply chain network by disrupting the normal operations of the Port of Baltimore, which is a major transportation hub for goods and people. The collapse of the bridge has created a challenge in the removal of the vessel, and the authorities have set a target date of May 10 for its removal. This indicates that the event has the potential to cause significant disruption to the supply chain network, and therefore, the impact assessment

In [41]:
eval(post_processing_result.content)

{'title': 'Vessel Removal to Take Place at Baltimore Port by May 10',
 'summary': "The Dali containership, which collided with the Francis Scott Key Bridge and has been lodged in the debris since March 26, is expected to be removed by May 10, according to officials involved in the clear-up efforts at Baltimore Port in the US. The removal is part of a larger operation that includes the recovery of two missing roadworkers and the reopening of the port's permanent channel by the end of May.",
 'impact': 'Yes',
 'reasoning': 'The event directly impacts the supply chain network by disrupting the normal operations of the Port of Baltimore, which is a major transportation hub for goods and people. The collapse of the bridge has created a challenge in the removal of the vessel, and the authorities have set a target date of May 10 for its removal. This indicates that the event has the potential to cause significant disruption to the supply chain network, and therefore, the impact assessment is 

## 3. LLAMA-3-8B-INSTRUCT

In [43]:
LLAMA3_8B_INSTRUCT_ENDPOINT = os.getenv("LLAMA3_8B_INSTRUCT_ENDPOINT")
LLAMA3_8B_INSTRUCT_API_KEY = os.getenv("LLAMA3_8B_INSTRUCT_API_KEY")
ENDPOINT_URL = LLAMA3_8B_INSTRUCT_ENDPOINT + "/v1/chat/completions"

In [51]:
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url = ENDPOINT_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=LLAMA3_8B_INSTRUCT_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
)

/anaconda/envs/langchain-demo-env/lib/python3.11/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [45]:
NEWS_PARSING_PROMPT = """
You are an expert extraction algorithm, specialized in news analysis.
Your goal is to understand and parse out the news article content based on the user's instructions of the output schema.
Only ouput the result into the schema without generating any other information outside the schema.
{format_instructions}

news article content:
{news_content}
"""

prompt = ChatPromptTemplate.from_template(
    template=NEWS_PARSING_PROMPT,
    partial_variables = {
        "format_instructions": format_instructions # passing in the formatting instructions created earlier in place of "format_instructions" placeholder
    }
)


llm = chat_model
full_chain = {"news_content": lambda x: x["news_content"]} | prompt | llm
result = full_chain.invoke({"news_content": sample_text}) 

In [50]:
# we can see that the instruct model, despite having smaller size, is able to follow instruction better (although the quality of the output may not be very good)
# however, there are still unnecessary characters before and after the JSON output
result.content

'```\n{\n    "title": "Bridge collapse at Baltimore Port: Vessel removal planned for 10 May",\n    "summary": "The Dali containership will be removed from the collapsed Francis Scott Key Bridge at Baltimore Port by 10 May, according to officials. The removal will allow for the reopening of the Port\'s permanent channel.",\n    "impact": "Yes",\n    "reasoning": "The vessel\'s removal will directly impact the supply chain network by reopening the Port\'s permanent channel, allowing ships to navigate in and out of the port.",\n    "vessel_name": ["Dali", "Francis Scott Key Bridge"]\n}\n```'

In [52]:
NEWS_PARSING_PROMPT_MODIFIED = """
You are an expert extraction algorithm, specialized in news analysis.
Your goal is to understand and parse out the news article content based on the user's instructions of the output schema.
Only ouput the result into the schema without generating any other information outside the schema.
Do not output extra characters before or after the JSON output, such as ```, so that the result can be parsed to a proper JSON later, while ensuring the JSON format is correct.
{format_instructions}

news article content:
{news_content}
"""

prompt_modified = ChatPromptTemplate.from_template(
    template=NEWS_PARSING_PROMPT_MODIFIED,
    partial_variables = {
        "format_instructions": format_instructions # passing in the formatting instructions created earlier in place of "format_instructions" placeholder
    }
)


llm = chat_model
full_chain_modified = {"news_content": lambda x: x["news_content"]} | prompt_modified | llm
result_modified = full_chain.invoke({"news_content": sample_text}) 

In [54]:
# you can see that the model output is now in a cleaner JSON format
result_modified.content

'{\n    "title": "Baltimore Port Bridge Collapse: Vessel to be Removed by 10 May",\n    "summary": "The Dali containership remains lodged in the debris of the Francis Scott Key Bridge at Baltimore Port in the US, which collapsed on March 26. Officials plan to remove the vessel by 10 May, but the clearance operation is complicated and poses safety risks. The port\'s 50ft deep and 700ft wide channel is expected to reopen by the end of May.",\n    "impact": "Yes",\n    "reasoning": "The bridge collapse and subsequent vessel collision have the potential to disrupt the normal operations of the port, causing delays and impacting the movement of goods. The clearance operation is complicated and poses safety risks, which could further impact the port\'s operations.",\n    "vessel_name": ["Dali", "Francis Scott Key Bridge"]\n}'

In [55]:
eval(result_modified.content)

{'title': 'Baltimore Port Bridge Collapse: Vessel to be Removed by 10 May',
 'summary': "The Dali containership remains lodged in the debris of the Francis Scott Key Bridge at Baltimore Port in the US, which collapsed on March 26. Officials plan to remove the vessel by 10 May, but the clearance operation is complicated and poses safety risks. The port's 50ft deep and 700ft wide channel is expected to reopen by the end of May.",
 'impact': 'Yes',
 'reasoning': "The bridge collapse and subsequent vessel collision have the potential to disrupt the normal operations of the port, causing delays and impacting the movement of goods. The clearance operation is complicated and poses safety risks, which could further impact the port's operations.",
 'vessel_name': ['Dali', 'Francis Scott Key Bridge']}

In [47]:
# as the .with_structured_output is not implemented in langchain, we can try to implement it inside the prompt
news_prompt_structured = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm, specialized in news analysis."
            "Only extract relevant information from the text. "
            "return a json with the following structure, do not generate anything else besides the JSON. Please close the JSON brackets properly"
            """
            
                "title":"[compulsory] One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.",\
                'summary':'[compulsory] A short summary of the text, maximum 200 words',\
                'impact':'[compulsory] Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps: \
                                        If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes. \
                                        Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes. \
                                        If not or uncertain, such as general knowledge, good news, individual personnel events, project annoucement etc., answer No',\
                'impact':'[compulsory] The reasoning behind your impact assessment based on the impact reasoning step above. Explain why you think the event will (Yes) or will not (No) impact the supply chain network.',\
                'vessel_name':'[optional] The names of the marine vessels or container ships mentioned in the text, if any.'
            
            """
        ),
        ("human", "{user_input}"),
    ]
)
ENDPOINT_URL = LLAMA3_8B_INSTRUCT_ENDPOINT + "/v1/chat/completions"
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url = ENDPOINT_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=LLAMA3_8B_INSTRUCT_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
)
llm = chat_model
sample_text = body
news_runnable = news_prompt_structured | llm
body_structured = news_runnable.invoke({"user_input": sample_text})

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [48]:
eval(body_structured.content)

SyntaxError: '{' was never closed (<string>, line 1)